# Machine Learning Project: Key Detection
Authors: Zach Hayes, Matt Gaetano, Max Ivry

Course: CS3540


In [11]:
import pandas as pd
import numpy as np

### Gather Training and Testing Data
Data cleaning and feature extraction occured earlier in process. See clean.ipynb

In [12]:
def retrieve_data():
    train_df = pd.read_json("train.json", lines=True)
    test_df = pd.read_json("test.json", lines=True)

    X_train = train_df.iloc[:, :-1]
    y_train = train_df["target"]

    X_test = test_df.iloc[:, :-1]
    y_test = test_df["target"]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = retrieve_data()

C:\Users\Zachary\AppData\Local\Temp\ipykernel_7872\677521648.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  train_df = pd.read_json("train.json", lines=True)
C:\Users\Zachary\AppData\Local\Temp\ipykernel_7872\677521648.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  train_df = pd.read_json("train.json", lines=True)
C:\Users\Zachary\AppData\Local\Temp\ipykernel_7872\677521648.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. 

### Machine Learning Algorithms